# Introduction to skeleton operations

There is lots more documentation on these skeletons at [this location](https://allenswdb.github.io/microns-em/em-skeletons.html).

Define a function to load neuronal skeletons

In [ ]:
def load_neuron(root_id, client):
    from skeleton_plot.skel_io import load_mw
    cloud_dir = 'https://storage.googleapis.com/allen-minnie-phase3/minniephase3-emily-pcg-skeletons/minnie_all/v661/meshworks'
    df = client.materialize.views.single_neurons(pt_root_id=root_id).query()
    if len(df)==0:
        raise ValueError(f'No pre-computed skeleton for root id {root_id}')
    else:
        return load_mw(cloud_dir, f'{root_id}_{df.iloc[0]["id"]}.h5')

Load example neuron

In [ ]:
nrn = load_neuron(864691135777016928, client)

Visualize neuron in vtk (optional)

In [ ]:
skel_actor = trimesh_vtk.skeleton_actor(nrn.skeleton)

trimesh_vtk.render_actors([skel_actor])

Measure distance to root (and convert from nm to microns by dividing by 1,000)

In [ ]:
dtr = nrn.distance_to_root(
    nrn.anno.post_syn.mesh_index
) / 1_000

In [ ]:
# Copy the dataframe and add the distance to root information to each 

post_df = nrn.anno.post_syn.df
post_df['dist_to_root'] = dtr

# example of showing you can slice the dataframe by distance
post_df[ post_df['dist_to_root'] < 50 ]

Find a branch of the soma and get just the synapses on that branch:

In [ ]:
# Identify the children of the cell body (or root) of the neuron

branch_starts = nrn.child_index(
    nrn.root,
)

Compute the indices that are downstream (away from root) of that first child node

In [ ]:
branch_inds = nrn.downstream_of(branch_starts[0])

Filter the postsynaptic synapse table by this collection of indices (converting `branch_inds` to a boolean mask via `branch_inds.to_mesh_mask`)

In [ ]:
branch_syn_df = nrn.anno.post_syn.filter_query(
    branch_inds.to_mesh_mask
).df

# convert voxels to nanometers
import numpy as np
branch_syn_pts = np.vstack(branch_syn_df['ctr_pt_position']) * [4,4,40]

Visualize the synapses along this one branch in 3d

In [ ]:
from meshparty import trimesh_vtk

ska = trimesh_vtk.skeleton_actor(nrn.skeleton)
pca = trimesh_vtk.point_cloud_actor(branch_syn_pts, size=900, color=(1,0,0)) 

trimesh_vtk.render_actors([ska, pca])

Plot number of synapses as a function of distance from the soma:

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.histplot(
    dtr,
    bins=40,
)